In [1]:
import config
API_KEY = config.API_KEY
API_SECRET = config.API_SECRET

https://stackoverflow.com/questions/40023931/how-to-get-real-estate-data-with-idealista-api

In [2]:
import pandas as pd
import json
import urllib
import requests as rq
import base64

In [3]:
def get_oauth_token(): 
    url = "https://api.idealista.com/oauth/token"    
    apikey= API_KEY #sent by idealist
    secret= API_SECRET #sent by idealista
    apikey_secret = apikey + ':' + secret
    auth = str(base64.b64encode(bytes(apikey_secret, 'utf-8')))[2:][:-1]
    headers = {'Authorization' : 'Basic ' + auth,'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}
    params = urllib.parse.urlencode({'grant_type':'client_credentials'}) #,'scope':'read'
    content = rq.post(url,headers = headers, params=params)
    bearer_token = json.loads(content.text)['access_token']
    return bearer_token

In [4]:
def search_api(token, URL):  
    headers = {'Content-Type': 'Content-Type: multipart/form-data;', 'Authorization' : 'Bearer ' + token}
    content = rq.post(url, headers = headers)
    result = json.loads(content.text)
    return result

In [5]:
# values: es, it, pt
country = 'es' 
# search language for summary. values: es, it, pt, en, ca
locale = 'es' 
language = 'es' 
# items per page: 50 as maximum allowed
max_items = '50'
# values: sale, rent
operation = 'sale'
# values: homes, offices, premises, garages, bedrooms
property_type = 'homes'
# allowed values by property type:
# garages: distance, price, street, photos, publicationDate, modificationDate (rent only), weigh, pricedown
# premises: distance, price, street, photos, publicationDate, modificationDate, size, floor, ratioeurm2 (rent only), weigh, pricedown
# offices: distance, price, street, photos, publicationDate, modificationDate, size, floor, ratioeurm2, weigh, pricedown
# homes: distance, price, street, photos, publicationDate, modificationDate, size, floor, rooms, ratioeurm2 (sólo alquiler), weigh, pricedown
# rooms: distance, price, street, photos, publicationDate, modificationDate, floor
order = 'priceDown' 
# geographic coordinates (WGS84) (latitude,longitude)
center = '43.3773,-8.39852' 
# distance to center, in metres (ratio)
distance = '3000'
# values: asc, desc
sort = 'desc'
# 
maxPrice = '200000'
# numPage = 1

In [6]:
df = pd.DataFrame()
for i in range(1,10):
    numPage = str(i)
    url = ('https://api.idealista.com/3.5/'+country+
            '/search?operation='+operation+
            '&locale='+locale+
            '&maxItems='+max_items+
            '&order='+order+
            '&center='+center+
            '&distance='+distance+
            '&maxPrice='+maxPrice+
            '&propertyType='+property_type+
            '&sort='+sort+ 
            '&numPage='+numPage+
            '&language='+language)
    result = search_api(get_oauth_token(), url)
    df_temp = pd.json_normalize(result, record_path = ["elementList"])
    df = pd.concat([df, df_temp])

In [7]:
print(len(df))
df

450


,propertyCode,thumbnail,externalReference,numPhotos,floor,price,propertyType,operation,size,exterior,...,detailedType.typology,suggestedTexts.subtitle,suggestedTexts.title,labels,detailedType.subTypology,neighborhood,parkingSpace.hasParkingSpace,parkingSpace.isParkingSpaceIncludedInPrice,parkingSpace.parkingSpacePrice,newDevelopmentFinished
0,97358818,https://img3.idealista.com/blur/WEB_LISTING/0/...,126855,27,3,104000.0,flat,sale,95.0,True,...,flat,"Agra del Orzán - Ventorrillo, A Coruña",Piso en Calle Páramo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,97422652,https://img3.idealista.com/blur/WEB_LISTING/0/...,556710,26,3,104000.0,flat,sale,85.0,True,...,flat,"Agra del Orzán - Ventorrillo, A Coruña",Piso en Calle Páramo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,96363274,https://img3.idealista.com/blur/WEB_LISTING/0/...,B197,32,18,195000.0,flat,sale,109.0,True,...,flat,"Los Castros - Castrillón, A Coruña",Piso en Avenida de Oza,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,98951857,https://img3.idealista.com/blur/WEB_LISTING/0/...,08629,16,2,80000.0,flat,sale,60.0,False,...,flat,"Monte Alto - Zalaeta - Atocha, A Coruña",Piso,"[{'name': 'apartamentoType', 'text': 'Apartame...",NaN,NaN,NaN,NaN,NaN,NaN
4,99137849,https://img3.idealista.com/blur/WEB_LISTING/0/...,NaN,29,5,105000.0,flat,sale,76.0,False,...,flat,"Os Mallos, A Coruña","Piso en Ronda Outeiro, 144",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,96415135,https://img3.idealista.com/blur/WEB_LISTING/0/...,22/002,8,3,169000.0,flat,sale,86.0,True,...,flat,"Monte Alto - Zalaeta - Atocha, A Coruña",Piso en Ronda de Monte Alto,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,98538348,https://img3.idealista.com/blur/WEB_LISTING/0/...,6506C,24,8,200000.0,flat,sale,131.0,True,...,flat,"Los Castros - Castrillón, A Coruña",Piso en Avenida de Oza,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,98849452,https://img3.idealista.com/blur/WEB_LISTING/0/...,K4,11,4,175000.0,flat,sale,81.0,True,...,flat,"Los Castros - Castrillón, A Coruña",Piso en Juan Montes s/n,NaN,NaN,NaN,True,True,NaN,NaN
48,97422449,https://img3.idealista.com/blur/WEB_LISTING/0/...,22-022,18,1,153000.0,flat,sale,55.0,True,...,flat,"Agra del Orzán - Ventorrillo, A Coruña",Piso en Ronda de Outeiro,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.to_csv('idealista_search.csv')